In [1]:
from ris import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2020-08-10_17:43
E:\RIS\Staff Folders\Samuel\Requests\SH\Safety_Data_Viewer


In [2]:
#Database connection
cdb = pysqldb.DbConnect(server='DOTDEVRHPGSQL01', database='CRASHDATA', type='PG')



Additional database connection details required:
User name (crashdata):soge
Password (crashdata)········
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-08-10 17:43:14


In [3]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-08-10 17:43:15


### Initial Table Containing All NYSDOT Information for Study Stretch. 

In [5]:
"""

--SIP 1319----------------------------------------------------------------------------------------------------------

DROP TABLE IF EXISTS sam_crashes_1319; 

CREATE TABLE sam_crashes_1319 AS 


WITH data AS(

SELECT distinct nys_a.*
FROM public.nysdot_all nys_a
JOIN public.node lion
on nys_a.nodeid::int = lion.nodeid::int
WHERE nys_a.case_yr>= 2014 and nys_a.case_yr<=2018
and lion.nodeid::int in(SELECT nodeid 
                        FROM public.node
                        WHERE masterid in (SELECT masterid
                                           FROM public.node
                                           WHERE nodeid::int in (47444,100531,9031585,9031837)))
)

,data2 AS (

SELECT data.*, age
FROM data
LEFT JOIN (SELECT nys_v.crashid, age 
           FROM (SELECT distinct crashid
                 FROM public.nysdot_vehicle
                 WHERE age between 1 and 17
                 and veh_typ in ('5','6')
                 GROUP by crashid
                 HAVING count(crashid) = 1
                 ) nys_v
           JOIN public.nysdot_vehicle nys_v_age
           on nys_v.crashid = nys_v_age.crashid
           WHERE veh_typ in ('5','6')
          ) ages
on data.crashid = ages.crashid

)

SELECT * FROM
data2
WHERE case_num in (SELECT DISTINCT case_num
                   FROM (SELECT DISTINCT case_num, count(case_num)c_id,accd_type_int, count(DISTINCT accd_type_int) c_ati
                         FROM data2
                         WHERE age is not null
                         GROUP BY case_num, accd_type_int) x
                   WHERE c_id = c_ati);



"""

"\n\n--SIP 1319----------------------------------------------------------------------------------------------------------\n\nDROP TABLE IF EXISTS sam_crashes_1319; \n\nCREATE TABLE sam_crashes_1319 AS \n\n\nWITH data AS(\n\nSELECT distinct nys_a.*\nFROM public.nysdot_all nys_a\nJOIN public.node lion\non nys_a.nodeid::int = lion.nodeid::int\nWHERE nys_a.case_yr>= 2014 and nys_a.case_yr<=2018\nand lion.nodeid::int in(SELECT nodeid \n                        FROM public.node\n                        WHERE masterid in (SELECT masterid\n                                           FROM public.node\n                                           WHERE nodeid::int in (47444,100531,9031585,9031837)))\n)\n\n,data2 AS (\n\nSELECT data.*, age\nFROM data\nLEFT JOIN (SELECT nys_v.crashid, age \n           FROM (SELECT distinct crashid\n                 FROM public.nysdot_vehicle\n                 WHERE age between 1 and 17\n                 and veh_typ in ('5','6')\n                 GROUP by crashid\n    

### Initial Table Containing All Fatality Information for Study Stretch. 

In [6]:
"""
--Fatals---------------------------

DROP TABLE IF EXISTS sam_fatalities_1319; 

CREATE TABLE sam_fatalities_1319 AS 

SELECT DISTINCT nodeid 
FROM public.node
WHERE masterid in (SELECT masterid
                   FROM public.node
                   WHERE nodeid::int in (47444,100531,9031585,9031837))
"""

'\n--Fatals---------------------------\n\nDROP TABLE IF EXISTS sam_fatalities_1319; \n\nCREATE TABLE sam_fatalities_1319 AS \n\nSELECT DISTINCT nodeid \nFROM public.node\nWHERE masterid in (SELECT masterid\n                   FROM public.node\n                   WHERE nodeid::int in (47444,100531,9031585,9031837))\n'

### 1. Sip 1319
#### Created table in Pgsql containing all nysdot_all data for Sip: 1319 - 111TH ST to 108TH STT

In [7]:
sip_1319 = cdb.dfquery("""SELECT * FROM sam_crashes_1319""")

In [8]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-08-10 17:35:29


### Injury Summary

In [ ]:
inj_sum = cdb.dfquery("""



WITH data as(
SELECT "Type", sum("Total Injuries") "Total Injuries", sum("Severe Injuries") "Severe Injuries", sum("Fatalities") "Fatalities", sum("Severe Injuries"  + "Fatalities") as "KSI"
FROM (
    SELECT * FROM (
    --Injuries/Severity By Mode
    SELECT  CASE WHEN accd_type_int = 1 then '1. ped'
                 WHEN accd_type_int = 2 then '2. bicycle'
                 WHEN accd_type_int = 3 then '3. mvo' end as "Type"
        ,sum(num_of_inj) as "Total Injuries"
        ,sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text))) as "Severe Injuries"
        ,0 as "Fatalities"
    FROM sam_crashes_1319
    WHERE accd_type_int = 1
    GROUP BY  accd_type_int
    ORDER BY  accd_type_int
    )injuries

    UNION 

    SELECT * from (
    SELECT CASE WHEN pos = 'PD' then '1. ped'
                WHEN pos = 'BI' then '2. bicycle'
                WHEN pos in ('MO','PS','DR') then '3. mvo' end as "Type"
           ,0 as "Total Injuries"
           ,0 as "Severe Injuries"
           ,count(id_) as "Fatalities"
    FROM public.fatality_nycdot_current fat
    WHERE fat.nodeid::int in ( select * from sam_fatalities_1319)
    and date_part('year',acdate) between 2014 and 2018
    and pos = 'PD'
    GROUP BY pos
    )fatalities

) inj_sum
GROUP BY "Type"
)


SELECT * from (
SELECT *
FROM data

UNION

SELECT 'Total' as "Type", "Total Injuries","Severe Injuries", "Fatalities", "KSI" 
FROM (select 'Total' as Year, 
      sum(data."Total Injuries") "Total Injuries",
      sum(data."Severe Injuries") "Severe Injuries",
      sum(data."Fatalities") "Fatalities",
      sum(data."KSI") "KSI"
      from data
      ) tot
)inj_summary
ORDER BY "Type"

""")

In [ ]:
inj_sum

### Fatalities by Mode

In [ ]:
cdb.connect()

In [4]:
fat = cdb.dfquery("""

--Fatalities by mode-------------------------------------------------------------

SELECT CASE WHEN pos = 'PD' then 'ped'
            WHEN pos = 'BI' then 'bicycle'
            WHEN pos in ('MO','PS','DR') then 'mvo' end as "Type"
      ,count(case when date_part('year',acdate) between 2014 and 2018 then id_ end) as "2014-2018 (5 Years)"
      ,count(case when date_part('year',acdate)>=2014 and acdate < '01-01-2020'::date then id_ end) as "2014-01/01/2020 (7 Years)"
FROM public.fatality_nycdot_current fat
WHERE  fat.nodeid::int in (select nodeid from sam_fatalities_1319)
and pos = 'PD'
GROUP BY pos


""") 


Failure:

- Query run 2020-08-11 12:32:10.154000
	

--Fatalities by mode-------------------------------------------------------------

SELECT CASE WHEN pos = 'PD' then 'ped'
            WHEN pos = 'BI' then 'bicycle'
            WHEN pos in ('MO','PS','DR') then 'mvo' end as "Type"
      ,count(case when date_part('year',acdate) between 2014 and 2018 then id_ end) as "2014-2018 (5 Years)"
      ,count(case when date_part('year',acdate)>=2014 and acdate < '01-01-2020'::date then id_ end) as "2014-01/01/2020 (7 Years)"
FROM public.fatality_nycdot_current fat
WHERE  fat.nodeid::int in (select nodeid from sam_fatalities_1319)
and pos = 'PD'
GROUP BY pos





SystemExit: 

C:\Python27\ArcGIS10.4\lib\site-packages\IPython\core\interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
fat

### Non-Fatal Injuries By Severity 

In [ ]:
cdb.connect()

In [ ]:
non_fat_sev = cdb.dfquery("""

--NON-FATAL INJURIES BY SEVERITY---------------------------------------------------


WITH data as(
SELECT * 
FROM (  SELECT 
        '1. A' as Severity,
        sum(CASE WHEN accd_type_int = 1 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text)) end) Pedestrian,
        sum(CASE WHEN accd_type_int = 2 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text)) end) Bicyclist,
        sum(CASE WHEN accd_type_int = 3 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text)) end) "Motor Vehicle",
        sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text))) as "Total"
        FROM sam_crashes_1319
        WHERE accd_type_int = 1

        union

        SELECT 
        '2. B' as Severity,
        sum(CASE WHEN accd_type_int = 1 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text)) end) Pedestrian,
        sum(CASE WHEN accd_type_int = 2 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text)) end) Bicyclist,
        sum(CASE WHEN accd_type_int = 3 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text)) end) "Motor Vehicle",
        sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text))) as "Total"
        FROM sam_crashes_1319
        WHERE accd_type_int = 1

        union

        SELECT 
        '3. C' as Severity,
        sum(CASE WHEN accd_type_int = 1 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text)) end) Pedestrian,
        sum(CASE WHEN accd_type_int = 2 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text)) end) Bicyclist,
        sum(CASE WHEN accd_type_int = 3 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text)) end) "Motor Vehicle",
        sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text))) as "Total"
        FROM sam_crashes_1319
        WHERE accd_type_int = 1

        union


        SELECT
        '4. Unknown' as Severity,
        sum(case when accd_type_int = 1 and coalesce(length(ext_of_inj::text),0) != num_of_inj then (num_of_inj + num_of_fat) - coalesce(length(ext_of_inj::text),0) end) Pedestrian,
        sum(case when accd_type_int = 2 and coalesce(length(ext_of_inj::text),0) != num_of_inj then (num_of_inj + num_of_fat) - coalesce(length(ext_of_inj::text),0)  end) Bicyclist,
        sum(case when accd_type_int = 3 and coalesce(length(ext_of_inj::text),0) != num_of_inj then (num_of_inj + num_of_fat) - coalesce(length(ext_of_inj::text),0)  end) "Motor Vehicle",
        sum((num_of_inj + num_of_fat) - coalesce(length(ext_of_inj::text),0)) as "Total"
        FROM sam_crashes_1319
        WHERE accd_type_int = 1

        )sev
        ORDER BY severity
)


SELECT * 
FROM ( SELECT * 
       FROM data

       UNION 

       SELECT '5. Total' as Severity, Pedestrian, Bicyclist, "Motor Vehicle", "Total"
       FROM (SELECT 'Total' as Year
            ,sum(data.Pedestrian) Pedestrian
            ,sum(data.Bicyclist) Bicyclist
            ,sum(data."Motor Vehicle") "Motor Vehicle"
            ,sum(data."Total") "Total"
            FROM data
              ) tot
        )non_fat_inj
ORDER BY severity




""")

In [ ]:
non_fat_sev.fillna(0)

### Injuries by Year

In [ ]:
inj_year = cdb.dfquery("""


--Injuries by year---------------------------------------------------



WITH data as (
SELECT 
case_yr::text as Year
,coalesce(sum(CASE WHEN accd_type_int = 1 THEN num_of_inj end),0) "Pedestrian" 
,coalesce(sum(CASE WHEN accd_type_int = 2 THEN num_of_inj end),0) "Bicyclist"
,coalesce(sum(CASE WHEN accd_type_int = 3 THEN num_of_inj end),0) "Motor Vehicle"
FROM sam_crashes_1319
WHERE accd_type_int = 1
GROUP by case_yr
order by case_yr
)

SELECT * FROM ( SELECT data.*, "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total
                FROM data

                UNION 

                SELECT tot.*, "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total
                FROM (SELECT 'Total' as Year, 
                      sum(data."Pedestrian") "Pedestrian",
                      sum(data."Bicyclist") "Bicyclist",
                      sum(data."Motor Vehicle") "Motor Vehicle"
                      FROM data
                    ) tot
               )inj_year
ORDER BY year

""")


In [ ]:
inj_year

### Injuries By Time of Day By Type

In [ ]:
inj_tod = cdb.dfquery("""


--Injuries by time of day by type---------------------------------------------------


WITH data as (
SELECT 
CASE WHEN date_part('hour',accd_tme) = 0 and date_part('minute',accd_tme) = 0 THEN '9. Unknown'
     WHEN date_part('hour',accd_tme) between 0 and 2 THEN '1. 00:00-03:00' 
     WHEN date_part('hour',accd_tme) between 3 and 5 THEN '2. 03:00-06:00' 
     WHEN date_part('hour',accd_tme) between 6 and 8 THEN '3. 06:00-09:00' 
     WHEN date_part('hour',accd_tme) between 9 and 11 THEN '4. 09:00-12:00' 
     WHEN date_part('hour',accd_tme) between 12 and 14 THEN '5. 12:00-15:00' 
     WHEN date_part('hour',accd_tme) between 15 and 17 THEN '6. 15:00-18:00' 
     WHEN date_part('hour',accd_tme) between 18 and 20 THEN '7. 18:00-21:00' 
     WHEN date_part('hour',accd_tme) between 21 and 23 THEN '8. 21:00-24:00' END as Time,
coalesce(sum(CASE WHEN accd_type_int = 1 THEN num_of_inj END),0) "Pedestrian", 
coalesce(sum(CASE WHEN accd_type_int = 2 THEN num_of_inj END),0) "Bicyclist",
coalesce(sum(CASE WHEN accd_type_int = 3 THEN num_of_inj END),0) "Motor Vehicle"
FROM sam_crashes_1319
WHERE accd_type_int = 1
GROUP by CASE WHEN date_part('hour',accd_tme) = 0 and date_part('minute',accd_tme) = 0 THEN '9. Unknown'
     WHEN date_part('hour',accd_tme) between 0 and 2 THEN '1. 00:00-03:00' 
     WHEN date_part('hour',accd_tme) between 3 and 5 THEN '2. 03:00-06:00' 
     WHEN date_part('hour',accd_tme) between 6 and 8 THEN '3. 06:00-09:00' 
     WHEN date_part('hour',accd_tme) between 9 and 11 THEN '4. 09:00-12:00' 
     WHEN date_part('hour',accd_tme) between 12 and 14 THEN '5. 12:00-15:00' 
     WHEN date_part('hour',accd_tme) between 15 and 17 THEN '6. 15:00-18:00' 
     WHEN date_part('hour',accd_tme) between 18 and 20 THEN '7. 18:00-21:00' 
     WHEN date_part('hour',accd_tme) between 21 and 23 THEN '8. 21:00-24:00' END
ORDER BY case WHEN date_part('hour',accd_tme) = 0 and date_part('minute',accd_tme) = 0 THEN '9. Unknown'
     WHEN date_part('hour',accd_tme) between 0 and 2 THEN '1. 00:00-03:00' 
     WHEN date_part('hour',accd_tme) between 3 and 5 THEN '2. 03:00-06:00' 
     WHEN date_part('hour',accd_tme) between 6 and 8 THEN '3. 06:00-09:00' 
     WHEN date_part('hour',accd_tme) between 9 and 11 THEN '4. 09:00-12:00' 
     WHEN date_part('hour',accd_tme) between 12 and 14 THEN '5. 12:00-15:00' 
     WHEN date_part('hour',accd_tme) between 15 and 17 THEN '6. 15:00-18:00' 
     WHEN date_part('hour',accd_tme) between 18 and 20 THEN '7. 18:00-21:00' 
     WHEN date_part('hour',accd_tme) between 21 and 23 THEN '8. 21:00-24:00' END)

SELECT * FROM (SELECT data.*, 
                "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total,
                (round((("Pedestrian"+"Bicyclist"+"Motor Vehicle")::float/(SELECT sum(num_of_inj) FROM sam_crashes_1319)::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE time != '9. Unknown'

                UNION 


                SELECT * , "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total, 'N/A' as "Percent of Known Injuries"
                FROM data
                WHERE data.time != '8. Unknown/Indeterminate'
                and time = '9. Unknown'


                UNION 

                SELECT tot.*, "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total, 'N/A' as "Percent of Known Injuries"
                FROM (SELECT 'Total' as Time, 
                      sum(data."Pedestrian") "Pedestrian",
                      sum(data."Bicyclist") "Bicyclist",
                      sum(data."Motor Vehicle") "Motor Vehicle"
                      FROM data
                      ) tot
                )inj_tod
ORDER BY time


""")

In [ ]:
inj_tod

### Pedestrian Injuries By Control And Pedestrian Action

In [ ]:
ped_inj_con = cdb.dfquery("""


--Pedestrian Injuries By Control and Pedestrian Action---------------------------------------------------


WITH data as (
SELECT 
CASE WHEN TRAF_CNTL = '02' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '1. Signalized Intersection: Crossing WITH Signal'
     WHEN TRAF_CNTL = '02' AND PED_ACTN = '02' AND PED_LOC = '1' THEN '2. Signalized Intersection: Crossing Against Signal'
     WHEN TRAF_CNTL = '03' AND PED_LOC = '1' THEN '3. Stop-Controlled Intersection / Crosswalk'
     WHEN TRAF_CNTL = '01' AND PED_LOC = '1' THEN '4. Other Actions/Uncontrolled Intersection'
     WHEN PED_LOC = '2' THEN '5. Midblock'
     WHEN TRAF_CNTL = '20' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '6. Other Control Types'
     ELSE '8. Unknown/Indeterminate' END as " ", 
coalesce(sum(num_of_inj),0) "Number of Injuries"
FROM sam_crashes_1319
WHERE accd_type_int = 1
GROUP by CASE WHEN TRAF_CNTL = '02' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '1. Signalized Intersection: Crossing WITH Signal'
              WHEN TRAF_CNTL = '02' AND PED_ACTN = '02' AND PED_LOC = '1' THEN '2. Signalized Intersection: Crossing Against Signal'
              WHEN TRAF_CNTL = '03' AND PED_LOC = '1' THEN '3. Stop-Controlled Intersection / Crosswalk'
              WHEN TRAF_CNTL = '01' AND PED_LOC = '1' THEN '4. Other Actions/Uncontrolled Intersection'
              WHEN PED_LOC = '2' THEN '5. Midblock'
              WHEN TRAF_CNTL = '20' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '6. Other Control Types'
              ELSE '8. Unknown/Indeterminate' END

)

SELECT * FROM ( SELECT data.*, 
                (round((("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != '8. Unknown/Indeterminate')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != '8. Unknown/Indeterminate'

                UNION

                SELECT data.*, 'N/A' as "Percent of Known Injuries"
                FROM data
                WHERE data." " = '8. Unknown/Indeterminate'

                UNION

                SELECT 
                '7. Total Known' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , (round((sum("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != '8. Unknown/Indeterminate')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != '8. Unknown/Indeterminate'

                UNION

                SELECT 
                'Total' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , 'N/A' as "Percent of Known Injuries"
                FROM data
                ) ped_inj_con
ORDER BY " "

""")

In [ ]:
ped_inj_con

### Bicycle Injuries By Intersection Control and Bicyclist Action

In [ ]:
bi_inj_con = cdb.dfquery("""


--Bicycle Injuries By Intersection Control and Bicyclist Action---------------------------------------------------



WITH data as (
SELECT 
CASE WHEN TRAF_CNTL = '02' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '1. Signalized Intersection: Crossing WITH Signal'
     WHEN TRAF_CNTL = '02' AND PED_ACTN = '02' AND PED_LOC = '1' THEN '2. Signalized Intersection: Crossing Against Signal'
     WHEN TRAF_CNTL = '03' AND PED_LOC = '1' THEN '3. Stop-Controlled Intersection / Crosswalk'
     WHEN TRAF_CNTL = '01' AND PED_LOC = '1' THEN '4. Other Actions/Uncontrolled Intersection'
     WHEN PED_LOC = '2' THEN '5. Midblock'
     WHEN TRAF_CNTL = '20' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '6. Other Control Types'
     ELSE '8. Unknown/Indeterminate' END as " ", 
coalesce(sum(num_of_inj),0) "Number of Injuries"
FROM sam_crashes_1319
WHERE accd_type_int = 2 and accd_type_int = 1
GROUP by CASE WHEN TRAF_CNTL = '02' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '1. Signalized Intersection: Crossing WITH Signal'
     WHEN TRAF_CNTL = '02' AND PED_ACTN = '02' AND PED_LOC = '1' THEN '2. Signalized Intersection: Crossing Against Signal'
     WHEN TRAF_CNTL = '03' AND PED_LOC = '1' THEN '3. Stop-Controlled Intersection / Crosswalk'
     WHEN TRAF_CNTL = '01' AND PED_LOC = '1' THEN '4. Other Actions/Uncontrolled Intersection'
     WHEN PED_LOC = '2' THEN '5. Midblock'
     WHEN TRAF_CNTL = '20' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '6. Other Control Types'
     ELSE '8. Unknown/Indeterminate' END

)

SELECT * FROM ( SELECT data.*, 
                (round((("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != '8. Unknown/Indeterminate')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != '8. Unknown/Indeterminate'

                UNION

                SELECT data.*, 'N/A' as "Percent of Known Injuries"
                FROM data
                WHERE data." " = '8. Unknown/Indeterminate'

                UNION

                SELECT 
                '7. Total Known' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , (round((sum("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != '8. Unknown/Indeterminate')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != '8. Unknown/Indeterminate'

                UNION


                SELECT 
                'Total' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , 'N/A' as "Percent of Known Injuries"
                FROM data
                ) bi_inj_con
ORDER BY " " 

""")

In [ ]:
bi_inj_con

In [ ]:
mvo_inj_coll = cdb.dfquery("""


--Motor Vehicle Injuries By Collision Type---------------------------------------------------


WITH data as(
SELECT 
CASE WHEN collision_ in ('03','10') THEN '1. LEFT Turn'
     WHEN collision_ in ('05','06') THEN '2. Right Turn'
     WHEN collision_ = '01' THEN '3. Rear-End'
     WHEN collision_ = '04' THEN '4. Right-Angle'
     WHEN collision_ = '02' THEN '5. Sideswipe(Same Direction)'
     WHEN collision_ = '07' THEN '6. Head-on'
     WHEN collision_ = '08' THEN '7. Sideswipe(Opposite Direction)'
     WHEN collision_ = '09' THEN '8. Other Known'
     WHEN collision_ in ('00','ZZ') THEN 'Unknown' END as " ",
sum(num_of_inj) as "Number of Injuries"--, crashid
FROM sam_crashes_1319
WHERE accd_type_int = 3 and accd_type_int = 1
GROUP by CASE WHEN collision_ in ('03','10') THEN '1. LEFT Turn'
     WHEN collision_ in ('05','06') THEN '2. Right Turn'
     WHEN collision_ = '01' THEN '3. Rear-End'
     WHEN collision_ = '04' THEN '4. Right-Angle'
     WHEN collision_ = '02' THEN '5. Sideswipe(Same Direction)'
     WHEN collision_ = '07' THEN '6. Head-on'
     WHEN collision_ = '08' THEN '7. Sideswipe(Opposite Direction)'
     WHEN collision_ = '09' THEN '8. Other Known'
     WHEN collision_ in ('00','ZZ') THEN 'Unknown' END--, crashid
order by " "
)

SELECT * FROM ( SELECT data.*, 
                (round((("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != 'Unknown')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != 'Unknown'
                

                UNION

                SELECT data.*, 'N/A' as "Percent of Known Injuries"
                FROM data
                WHERE data." " = 'Unknown'

                UNION

                SELECT 
                '9. Total Known' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , (round((sum("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != 'Unknown')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != 'Unknown'

                UNION

                SELECT 
                'Total' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , 'N/A' as "Percent of Known Injuries"
                FROM data
                ) ped_inj_con
ORDER BY " " 


"""
                          )

In [ ]:
mvo_inj_coll

### Injuries By Age Group

In [ ]:
cdb.connect()

In [ ]:
inj_age = cdb.dfquery("""


--Injuries By Age GROUP---------------------------------------------------


WITH data as(
SELECT 
CASE WHEN ages.age::int between 1 and 17 and num_of_inj = 1 THEN '1. Children(1-17)'
     WHEN ages.age::int between 18 and 29 and num_of_inj = 1 THEN '2. Young Adults(18-29)'
     WHEN ages.age::int between 30 and 64 and num_of_inj = 1 THEN '3. Adults(30-64)'
     WHEN ages.age::int between 65 and 120 and num_of_inj = 1 THEN '4. Seniors(65-120)'
     WHEN ages.age is null or num_of_inj!=1 THEN '5. Unknown' End "Age GROUP",
coalesce(sum(case when accd_type_int = 1 then num_of_inj end),0) "Pedestrian", 
coalesce(sum(case when accd_type_int = 2 then num_of_inj end),0) "Bicyclist" 
FROM sam_crashes_1319
LEFT JOIN (SELECT nys_v.crashid, age 
           FROM (SELECT distinct crashid
                 FROM public.nysdot_vehicle
                 WHERE age between 1 and 120
                 and veh_typ in ('5','6')
                 GROUP by crashid
                 HAVING count(crashid) =1
                 ) nys_v
           JOIN public.nysdot_vehicle nys_v_age
           on nys_v.crashid = nys_v_age.crashid
           WHERE veh_typ in ('5','6')
          ) ages
on sam_crashes_1319.crashid = ages.crashid
WHERE accd_type_int = 1
GROUP by CASE WHEN ages.age::int between 1 and 17 and num_of_inj = 1 THEN '1. Children(1-17)'
              WHEN ages.age::int between 18 and 29 and num_of_inj = 1 THEN '2. Young Adults(18-29)'
              WHEN ages.age::int between 30 and 64 and num_of_inj = 1 THEN '3. Adults(30-64)'
              WHEN ages.age::int between 65 and 120 and num_of_inj = 1 THEN '4. Seniors(65-120)'
              WHEN ages.age is null or num_of_inj!=1 THEN '5. Unknown' End
ORDER BY "Age GROUP"
)

SELECT * FROM ( SELECT data.*, "Pedestrian"+"Bicyclist" Total
                FROM data

                UNION 

                SELECT tot.*, "Pedestrian"+"Bicyclist" Total
                FROM (SELECT 'Total' as Year 
                     ,sum(data."Pedestrian") "Pedestrian"
                     ,sum(data."Bicyclist") "Bicyclist"
                     FROM data
                      ) tot

               ) inj_age
                
ORDER BY "Age GROUP"

""")

In [ ]:
inj_age

In [ ]:
cdb.connect()

### Ped Injuries By Veh Action And Pre Action

In [ ]:
ped_veh_actn = cdb.dfquery("""


WITH data as(
SELECT 
CASE WHEN pre_accd_actn = '03' THEN '1. Left Turn'
     WHEN pre_accd_actn = '02' THEN '2. Right Turn'
     WHEN pre_accd_actn = '01' THEN '3. Going Straight'
     WHEN pre_accd_actn = '04' THEN '4. Making U Turn'
     WHEN pre_accd_actn = '15' THEN '5. Backing'
     WHEN pre_accd_actn not in ('01','02','03','04','15','??','YY','ZZ') THEN '6. Other'   
     WHEN pre_accd_actn in ('??','YY','XX', 'ZZ') or single_veh.crashid is null THEN '7. Unknown' END as " ",
coalesce(sum(CASE WHEN ped_actn = '01' THEN num_of_inj END),0) as "Crossing WITH Signal",
coalesce(sum(CASE WHEN ped_actn = '02' THEN num_of_inj END),0) as "Crossing Against Signal",
coalesce(sum(CASE WHEN ped_actn = '03' THEN num_of_inj END),0) as "Crossing No Signal Marked Crosswalk",
coalesce(sum(CASE WHEN ped_actn = '04' THEN num_of_inj END),0) as "Crossing No Signal or Crosswalk",
coalesce(sum(CASE WHEN ped_actn not in ('01','02','03','04','??','YY','XX','ZZ') THEN num_of_inj END),0) as "Others",
coalesce(sum(CASE WHEN ped_actn in ('??','YY','XX', 'ZZ') THEN num_of_inj END),0) as "UnKnown"
FROM sam_crashes_1319
LEFT JOIN (SELECT distinct crashid
         FROM public.nysdot_vehicle
         WHERE veh_typ = '6'
         GROUP by crashid
         HAVING count(crashid) = 1) single_ped
     on sam_crashes_1319.crashid = single_ped.crashid
LEFT JOIN (SELECT sv.crashid, sv2.pre_accd_actn
       FROM (SELECT distinct crashid--, pre_accd_actn
                 FROM public.nysdot_vehicle
                 WHERE veh_typ not in ('5','6')
                 GROUP by crashid--, pre_accd_actn
                 HAVING count(crashid) = 1)sv
         LEFT JOIN (SELECT distinct crashid, pre_accd_actn
               FROM public.nysdot_vehicle
               WHERE veh_typ not in ('5','6')
               --WHERE pre_accd_actn not in ('??','YY','XX')
               ) sv2
            on sv.crashid = sv2.crashid) single_veh
    on sam_crashes_1319.crashid = single_veh.crashid
WHERE accd_type_int = 1
GROUP by CASE WHEN pre_accd_actn = '03' THEN '1. Left Turn'
          WHEN pre_accd_actn = '02' THEN '2. Right Turn'
              WHEN pre_accd_actn = '01' THEN '3. Going Straight'
          WHEN pre_accd_actn = '04' THEN '4. Making U Turn'
          WHEN pre_accd_actn = '15' THEN '5. Backing'
          WHEN pre_accd_actn not in ('01','02','03','04','15','??','YY','ZZ') THEN '6. Other'     
          WHEN pre_accd_actn in ('??','YY','XX', 'ZZ') or single_veh.crashid is null THEN '7. Unknown' END
)


SELECT * FROM ( SELECT data.*, "Crossing WITH Signal" + "Crossing Against Signal" + "Crossing No Signal Marked Crosswalk" + "Crossing No Signal or Crosswalk" +  "Others" + "UnKnown" Total
        FROM data

        UNION 

        SELECT tot.*, "Crossing WITH Signal" + "Crossing Against Signal" + "Crossing No Signal Marked Crosswalk" + "Crossing No Signal or Crosswalk" +  "Others" + "UnKnown" Total
        FROM (SELECT 'Total' as " ", 
              sum(data."Crossing WITH Signal") "Crossing WITH Signal",
              sum(data."Crossing Against Signal") "Crossing Against Signal",
              sum(data."Crossing No Signal Marked Crosswalk") "Crossing No Signal Marked Crosswalk",
              sum(data."Crossing No Signal or Crosswalk") "Crossing No Signal or Crosswalk",
              sum(data."Others") "Others",
              sum(data."UnKnown") "UnKnown"
              FROM data
            ) tot

        ) ped_veh_actn
ORDER BY " "



""")

In [ ]:
ped_veh_actn